## Create a sample set to generate a dataset for fine tuning.

First load the FACTors data

In [4]:
import pandas as pd

# Load the data
factors_df = pd.read_csv("Data/FACTors.csv")

# Identify article_ids that occur only once
article_counts = factors_df['article_id'].value_counts()
duplicate_article_ids = article_counts[article_counts > 1]
unique_article_ids = article_counts[article_counts == 1].index

# Filter the DataFrame to keep only unique article_ids
clean_factors_df = factors_df[factors_df['article_id'].isin(unique_article_ids)]

# Confirm removal
print(f"Original rows: {len(factors_df)}")
print(f"Articles with multiple claims: {len(duplicate_article_ids)}")
print(f"Rows after removing duplicates: {len(clean_factors_df)}")

Original rows: 118112
Articles with multiple claims: 12
Rows after removing duplicates: 117981


## Build a dataset with claims and factchecked answers
Retrieve first a sample of 1000 claims and fact checked articles, make sure to divide the verdicts equally

In [5]:
# Take a subset of the largest fact checking organisations
factors_sub_df=clean_factors_df[clean_factors_df["organisation"].isin(["PolitiFact", "AFP Fact Check", "Snopes", "WebQoof", "FactCheck.org"])]
factors_sample_df= factors_sub_df.sample(n=1000, random_state=12)

Retrieve the full articles fromt the url

In [6]:
factors_sample_df=factors_sample_df[['claim','url']]
factors_sample_df.head(10)

,claim,url
81368,"""Arizona officials caught changing ballots, ha...",https://www.politifact.com/factchecks/2024/nov...
90708,The Yeti snow monster from Disneyland's iconic...,https://www.snopes.com/fact-check/disney-yeti/
67021,"""I can tell you that the enhanced interrogatio...",https://www.politifact.com/factchecks/2016/may...
10591,Nigerian election tribunal witness goes on the...,https://factcheck.afp.com/doc.afp.com.33NE7Y8
75871,"""We essentially repealed Obamacare because we ...",https://www.politifact.com/factchecks/2017/dec...
71583,"President Obama plans to ""impose a tax of at l...",https://www.politifact.com/factchecks/2011/nov...
75185,"""Almost half a million people are still eligib...",https://www.politifact.com/factchecks/2016/aug...
91346,Two 'racist' Black teenagers shot and killed a...,https://www.snopes.com/fact-check/thugs-shoot-...
71546,"Says Barack Obama had ""huge majorities"" in Con...",https://www.politifact.com/factchecks/2011/dec...
91280,"Walter ""Blackie"" Wetzel, a former leader of th...",https://www.snopes.com/fact-check/walter-wetze...


### First step: create a summary and listing possible problems
Retrieve information and create a summary as done in the original workflow of the assistant for these 1000 claims.

In [7]:
get_information_prompt = """
### Role
You are a neutral, guiding assistant that helps students through the fact-checking process step by step. Your main goal is not to provide answers, 
but to support the student in developing their own reasoning and critical thinking. You do this by asking open, 
reflective questions that encourage exploration, justification, and evaluation. You do not take over the student's thinking, 
and you do not complete tasks for them. Avoid giving conclusions or definitive judgments unless the workflow specifically requires it.

In this step your are tasked with extracting detailed information about a claim to determine its checkability.

### Claim
{claim}

### Important Rules
This part focuses on determining whether the subject is clear, the claim is quantitative, how precise it is, how the data was derived, 
and what additional details are present or missing. 
You don't need to acquire all missing details right now; just identify what is missing and formulate one clarifying question. 
If the user says no more details are available, proceed with what you have.

### Steps
1. Identify the subject. If unclear → "unclear".
2. Determine if the claim is *quantitative*. Set *quantitative* to true/false.
3. Assess precision: "precise", "vague", or "absolute (100%)". If qualitative, use "".
4. Identify what the claim is *based on* (e.g., "survey …", "official statistics"). If none → "unclear".
5. Briefly *explain your reasoning* (quote/phrase from the claim).
6. Ask exactly one *clarifying/confirmation question* that would make the claim checkable.
7. Identify *alerts/warnings*: unclear subject, qualitative claim, vague quantitative claim, geography missing, time period missing, methodological details absent. 
Don't mention an alert when the information is present.
8. *Summarize concisely* what is currently known about the claim and its checkability.
   - Include: subject, type (quantitative/qualitative), precision, basis, and uncertainties.
   - Mention any active alerts or missing information.

Keep your tone neutral and analytical.

### Output Format

Return a single JSON object with exactly these fields:

- "subject": string. Use "unclear" if the subject is not clear.
- "quantitative": string. Start with "true" or "false", followed by a short explanation.
- "precision": string. One of "precise", "vague", "absolute (100%)", or "" (empty string), plus a short explanation.
- "based_on": string. Either a brief description of the methodology/source or "unclear", plus a short explanation.
- "question": string. One open clarifying or confirmation question; don't ask for specific details.
- "alerts": array of strings. Each alert as a short string; use [] if none.
- "summary": string. A concise summary of the claim and its checkability status.

The response must be valid JSON and contain **only** this JSON object, with no extra text before or after it.

### Examples
Example A (qualitative):
{{
  "subject": "Spanish court sentencing of Catalan leaders (2019)",
  "quantitative": "false, because there is no quantitative data",
  "precision": "precise, because it refers to a specific legal event in a defined time and place",
  "based_on": "news reporting / legal documents, because the information is typically drawn from official court rulings and journalistic coverage",
  "question": "What is the main point you are trying to understand here?",
  "alerts": ["qualitative claim", "methodological details absent", "geography present", "time period present"],
  "summary": "A qualitative claim about a specific legal event; methodology implied but not fully detailed."
}}

Example B (quantitative but vague):
{{
  "subject": "EU asylum applications",
  "quantitative": "true, because it refers to measurable counts of applications",
  "precision": "vague, because no time frame, comparison, or dataset is identified",
  "based_on": "unclear, because the data source could vary (Eurostat, UNHCR, national agencies, media summaries)",
  "question": "What do you think is important to clarify before evaluating this?",
  "alerts": ["vague quantitative claim", "time period missing", "source/methodology missing", "geography: EU (present)"],
  "summary": "A quantitative claim lacking precision and methodological details; several key elements are missing for checkability."
}}
"""

In [8]:
import pandas as pd
from langchain_core.messages import SystemMessage, HumanMessage
import tqdm as notebook_tqdm
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from typing_extensions import List

load_dotenv(dotenv_path=".env", override=True)

class MoreInfoResult(BaseModel):
    subject: str = Field("", description="The subject of the claim")
    quantitative: str = Field("", description="Is the claim quantitative?")
    precision: str = Field("", description="How precise is it?")
    based_on: str = Field("", description="how was the data collected or derived?")
    question: str = Field("", description="Question to user for clarification if needed")
    alerts: List[str] = Field([], description="Any alerts or warnings about the claim")
    summary: str = Field("", description="A concise summary of the claim")

#low temperature for more factual answers, 
llmQwen = ChatGroq(model_name="qwen/qwen3-32b", temperature=0.1)
llmGPT5 = ChatOpenAI(model="gpt-5", temperature=0.1)

c:\temp\checkmate\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import json

def retrieve_info(claim: str) -> dict:
    """Gather more information about a potentially checkable claim."""

    # Let LangChain handle structured output using tools/schema
    structured_llm = llmGPT5.with_structured_output(MoreInfoResult)  # 👈 no method="json_mode"

    # You can keep your prompt, but see step 2 below to simplify the JSON part
    prompt = get_information_prompt.format(claim=claim)

    # Call the model – pass the prompt string, not a list of HumanMessage
    result = structured_llm.invoke(prompt)

    # Pydantic model → Python dict (perfect for pandas)
    return result.model_dump()

In [10]:
print(retrieve_info("The speed limit does not save a lot of CO₂"))

{'subject': 'Speed limit policy’s impact on CO₂ emissions', 'quantitative': 'true, because it asserts a magnitude of CO₂ savings (“does not save a lot”), which is measurable', 'precision': 'vague, because “not a lot” provides no numeric estimate, threshold, baseline, geography, or timeframe', 'based_on': 'unclear, because no source or method is cited; could rely on transport models, emissions inventories, or policy evaluations', 'question': 'What context are you thinking of (e.g., country/road types/timeframe) and how would you recognize that the CO₂ savings are “a lot” versus “not a lot”?', 'alerts': ['vague quantitative claim', 'geography missing', 'time period missing', 'methodological details absent', 'policy specifics missing'], 'summary': 'Claim concerns the effect of a (unspecified) speed limit on CO₂ emissions; it is quantitative but vague. No source or method is provided. Key uncertainties include context (geography, road types), timeframe, policy parameters (which speed limit

In [ ]:
test_df = factors_sample_df.head(5).copy()
test_df["analysis"] = test_df["claim"].apply(retrieve_info)
test_analysis_df = test_df["analysis"].apply(pd.Series)
test_all_df = pd.concat([test_df, test_analysis_df], axis=1)
#factors_sample_df["analysis"] = factors_sample_df["claim"].apply(retrieve_info)
#factor_analysis_df = factors_sample_df["analysis"].apply(pd.Series)
#factors_all_df = pd.concat([factors_sample_df, factor_analysis_df], axis=1)

#factors_all_df.to_csv("Data/finetune_data_1.csv", index=False)
#factors_all_df.head(10)
test_all_df.head(10)

use GPT5, often regarded as best model for various tasks, including language tasks:
- https://artificialanalysis.ai/leaderboards/models
- https://www.vellum.ai/llm-leaderboard?utm_source=google&utm_medium=organic
- https://www.shakudo.io/blog/top-9-large-language-models

## Create JSONL messages for finetuning
Next, create messages containing a claim, a verdict, and an explanation, then add Socratic questions to encourage critical thinking and reflection.

In [ ]:
from pathlib import Path
import pandas as pd
import re
import json
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
import tqdm as notebook_tqdm
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

#low temperature for more factual answers,
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.2 )

SYS = """You are given a fact-check CLAIM and its justification as SHORT_EXPLANATION explaining why it is labeled as true, false, mostly true, 
mostly false, or uncheckable. Your task is to generate five Socratic questions that probe the justification and verdict. The goal is to 
challenge the reasoning, surface blind spots, and encourage deeper reflection, not to accept the explanation at face value. Since the output
 will be used to finetune an LLM that critiques the reasoning of a fact-checking model, ensure that your questions reflect the following principles:
- Factuality – Do the claims rely on verifiable evidence? Could missing or weak evidence be questioned?
- Objectivity – Is the reasoning neutral, or does it show bias? How could the framing be challenged?
- Fairness – Are multiple perspectives considered? Is the reasoning applied consistently?
- Transparency – Is the explanation clear about its sources and reasoning steps? What is hidden or assumed?
- Hallucinations – Does the explanation risk introducing unsupported or invented information?
- Strategies & Alternatives – Are there other ways to frame, investigate, or reason about the claim?

When writing questions, draw from the following categories of Socratic questioning. Use them as inspiration to diversify your five questions 
(do not stick to just one category):

Purpose – probe the aim or agenda.
- What is your purpose right now?
- Why are you writing this?
- What do you want to persuade them of?
- What is our central aim or task in this line of thought?

Questions – probe the underlying questions.
- I am not sure exactly what question you are raising. Could you explain it?
- Is this question the best one to focus on, or is there a more pressing one?
- What questions might we be failing to ask that we should be asking?

Information – probe the evidence or data.
- On what information are you basing that comment?
- How do we know this information is accurate? How could we verify it?
- Have we failed to consider any information or data we need to consider?

Inferences & Conclusions – probe how the conclusion was drawn.
- How did you reach that conclusion?
- Could you explain your reasoning?
- Is there an alternative plausible conclusion?

Concepts & Ideas – probe key ideas being applied.
- What is the main idea you are using in your reasoning?
- Are we using the appropriate concept, or do we need to reconceptualize the problem?
- Do we need more facts, or do we need to rethink how we are labeling the facts?

Assumptions – probe what is taken for granted.
- What exactly are you taking for granted here?
- Why are you assuming that? Shouldn’t we rather assume that…?
- What alternative assumptions might we make?

Implications & Consequences – probe what follows.
- What are you implying when you say…?
- If we do this, what is likely to happen as a result?
- Have you considered the implications of this reasoning?
- Viewpoints & Perspectives – probe alternative frames.

From what point of view are you looking at this?
- Is there another point of view we should consider?
- Which of these possible viewpoints makes the most sense given the situation?

Instructions:
- Do not repeat the justification.
- Do not state whether the verdict is correct.
- Ask probing questions that challenge the reasoning, highlight blind spots, and open space for reconsideration.
- Ensure the five questions you generate come from different categories where possible

Output format (JSONL):
{
  "claim": "the original claim",
  "short_explanation": "the original short explanation",
  "verdict": "The verdict as written in the explanation: true, false, mostly true, mostly false or uncheckable",
  "questions": [
    "What is our central aim or task in this line of thought?",
    "What is the underlying question that this explanation is really trying to address?",
    "How do we know this information is accurate, and how could we verify it?",
    "Is there an alternative plausible conclusion based on the same reasoning?",
    "Is there another point of view we should consider when evaluating this claim?"
  ]
}
"""

def add_questions(claim: str, short_explanation: str):
    msgs = [
        SystemMessage(content=SYS),
        HumanMessage(content=f'CLAIM: {claim}\nSHORT_EXPLANATION: {short_explanation}')
    ]
    try:
        resp = llm.invoke(msgs)
        text = getattr(resp, "content", str(resp)).strip()
        one_line = " ".join(text.split())

        return one_line
    except Exception:
        return None

# --- Load data and compute short_explanation as before ---
factchecks_df = pd.read_csv("Data/factchecks_with_verdicts.csv")

# --- Generate JSONL lines and write them to a single file ---
output_path = Path("Data/socratic_questions.jsonl")
output_path.parent.mkdir(parents=True, exist_ok=True)

valid_lines = []

for _, row in factchecks_df.iterrows():
    line = add_questions(row["claim"], row["short_explanation"])
    if not line:
        continue

    try:
        obj = json.loads(line)  # parse the JSON string
    except json.JSONDecodeError:
        continue  # skip if the model output was not valid JSON

    # Expand into one object per question
    for q in obj.get("questions", []):
        new_obj = {
            "claim": obj["claim"],
            "short_explanation": obj["short_explanation"],
            "verdict": obj["verdict"],
            "question": q,
        }
        valid_lines.append(new_obj)

with output_path.open("w", encoding="utf-8") as f:
    for obj in valid_lines:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print(f"Wrote {len(valid_lines)} JSON objects to {output_path}")

Wrote 4995 JSON objects to Data\socratic_questions.jsonl
